In [1]:
import pandas as pd
import numpy as np
import math
import datetime
from scipy import stats
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [2]:
pwd

'/Users/irfanhilman/factor_analysis'

In [4]:
database_1 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 1.csv')
database_2 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 2.csv')
database_3 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 3.csv')
database_4 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 4.csv')
database_5 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 5.csv')
database_6 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 6.csv')
database_7 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 7.csv')
database_8 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 8.csv')
database_9 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 9.csv')
database_10 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 10.csv')
database_11 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 11.csv')
database_12 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 12.csv')
database_13 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 13.csv')
database_14 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 14.csv')
database_15 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 15.csv')
database_16 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 16.csv')
database_17 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 17.csv')
database_18 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 18.csv')
database_19 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 19.csv')
database_20 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 20.csv')
database_21 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 21.csv')
database_22 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 22.csv')
database_23 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 23.csv')
database_24 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 24.csv')
database_25 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 25.csv')
database_26 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 26.csv')
database_27 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 27.csv')
database_28 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 28.csv')
database_29 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 29.csv')
database_30 = pd.read_csv('/Users/irfanhilman/quanti-trading-plan/data/database_daily_update/Database Part 30.csv')
database = pd.concat([database_1,database_2,database_3,database_4,database_5,database_6,database_7,database_8,database_9,database_10,
                      database_11,database_12,database_13,database_14,database_15,database_16,database_17,database_18,database_19,database_20,
                      database_21,database_22,database_23,database_24,database_25,database_26,database_27,database_28,database_29,database_30])
database = database.drop(columns='Unnamed: 0')
#Substitusi nilai Open, High, Low Price yg 0 dengan Close Price
database['Open Price'] = np.where(database['Open Price'].eq(0),database['Close Price'],database['Open Price'])
database['High Price'] = np.where(database['High Price'].eq(0),database['Close Price'],database['High Price'])
database['Low Price'] = np.where(database['Low Price'].eq(0),database['Close Price'],database['Low Price'])

In [5]:
#BASIC PARAMETER
start_date = '2010-01-01'
end_date = '2025-03-21' #'2025-03-21'
hold_period = 21 #in days
factor = ['Sales/Market','Momentum 252 Days Skip 0 Days' ] #other than Size (Market Cap)
weighting = 'Value-Weight' #'Value-Weight','Equal-Weight'
hold_type = 'Periodic' #'Periodic' or 'First-Day in Month'
#partition = 10 

In [6]:
#SCREENING PARAMETER
screening_param = ['Market Cap','Total Ekuitas','Close Price','Median Transaction Value 1 Month'] #Params
screening_sign = ['Above','Above','Above','Above'] #Above, Below, Not Equal, or Between. If Between, assign a list in screening_val: screening_val = [[min,max]]
screening_val = [100_000_000_000,0,50,100_000_000] #Numeric variable


In [7]:
#Drop Stocks
drop_list = ['excl', 'ihsg']

In [8]:
def drop_stocks(data,drop_list):
    for r in drop_list:
        data= data[data['Kode']!=r]
    else: pass
    return data

def screening_stocks(data,screening_params,screening_signs,screening_vals):
    if len(screening_params) == len(screening_signs) == len(screening_vals):
        for ens,j in enumerate(screening_params):  #Initial Screening
            if screening_signs[ens] == 'Above':
                data = data[data[j]>screening_vals[ens]]
            elif screening_signs[ens] == 'Below':
                data = data[data[j]<screening_vals[ens]]
            elif screening_signs[ens] == 'Not Equal':
                data = data[data[j]!=screening_vals[ens]]
            elif screening_signs[ens] == 'Between':
                data = data[(data[j]>=screening_vals[ens][0])&(data[j]<=screening_vals[ens][1])]
            else:
                raise TypeError('screening_sign wrong. Select either Above or Below')
    else:
        raise Exception('Number of elements in screening parameter list are different')
    return data

def apply_weighting(data,weighting):
        data = data.dropna(subset='Market Cap')
        if weighting=='Equal-Weight':
            data['Weight'] = 1/len(data)
        elif weighting=='Value-Weight':
            data['Weight'] = data['Market Cap']/data['Market Cap'].sum()
        else:
            raise TypeError('weighting wrong. See notes in the parameter setting section')
        return data

Check buy_date_list dan sell_date_list beda length, harusnya sell_date_list bisa tambah 1 element

In [220]:
database_date_list = list(database['Date'].sort_values().unique())          
if hold_type =='Periodic': 
    selected_date_list = database_date_list[::hold_period+1]
elif hold_type =='First-Day in Month':
    selected_date_list = []
    database['Year'] = database['Date'].str[:4].astype('int')
    database['Month'] = database['Date'].str[5:7].astype('int')
    for i in database['Year'].unique():
        database_i = database[database['Year']==i]
        for j in database_i['Month'].unique():
            database_j = database_i[database_i['Month']==j]
            database_j = database_j.sort_values(by='Date')
            date_list_j = list(database_j['Date'].unique())
            first_day = date_list_j[0]
            selected_date_list.append(first_day)
    database = database.drop(columns=['Year','Month'])
else:
    raise TypeError('hold_type wrong. Select "Periodic" or "First-Day in Month"')
buy_date_list = [d for d in selected_date_list if d >= start_date and d <= end_date]
sell_date_list = []
for i in buy_date_list[1:]:
    sell_date_i = database_date_list.index(i)-1
    sell_date = database_date_list[sell_date_i]
    sell_date_list.append(sell_date)
param_date_index = [(database_date_list.index(p)-1) for p in buy_date_list]
param_date_list = [database_date_list[p] for p in param_date_index]
size_param = ['Date','Kode','Market Cap']
database_part = database.copy()
database_part = database_part[database_part['Date'].isin(param_date_list)]
database_part = database_part.dropna(subset=['Date','Kode','Market Cap'])
database_part = database_part.dropna(subset=factor)
database_part = drop_stocks(data=database_part,drop_list=drop_list)
database_part = screening_stocks(data=database_part,screening_params=screening_param,screening_signs=screening_sign,screening_vals=screening_val)
database_part_size = database_part[size_param]
df_small_list = []
df_big_list = []
for i in list(database_part_size['Date'].sort_values().unique()):
    database_part_size_i = database_part_size[database_part_size['Date']==i]
    database_part_size_i = database_part_size_i.sort_values(by='Market Cap')
    df_small_i = database_part_size_i[database_part_size_i['Market Cap']<database_part_size_i['Market Cap'].median()]
    df_big_i = database_part_size_i[database_part_size_i['Market Cap']>=database_part_size_i['Market Cap'].median()]
    df_small_list.append(df_small_i)
    df_big_list.append(df_big_i)
df_small = pd.concat(df_small_list)
df_big = pd.concat(df_big_list)
df_factor_top_dic = {}
df_factor_medium_dic = {}
df_factor_bottom_dic = {}
for f in factor:
    df_factor_top_dic[f"Top {f}"] = []
    df_factor_medium_dic[f"Medium {f}"] = []
    df_factor_bottom_dic[f"Bottom {f}"] = []
for f in factor:
    for i in list(database_part['Date'].sort_values().unique()):
        database_part_i = database_part[database_part['Date']==i]
        database_part_factor_i = database_part_i[['Date','Kode','Market Cap',f]]
        database_part_factor_i = database_part_factor_i.sort_values(by=f)
        database_part_factor_i_top =database_part_factor_i[database_part_factor_i[f]>database_part_factor_i[f].quantile(0.7)]
        database_part_factor_i_medium =database_part_factor_i[(database_part_factor_i[f]>=database_part_factor_i[f].quantile(0.3))&(database_part_factor_i[f]<=database_part_factor_i[f].quantile(0.7))]
        database_part_factor_i_bottom =database_part_factor_i[database_part_factor_i[f]<database_part_factor_i[f].quantile(0.3)]
        df_factor_top_dic[f"Top {f}"].append(database_part_factor_i_top)
        df_factor_medium_dic[f"Medium {f}"].append(database_part_factor_i_medium)
        df_factor_bottom_dic[f"Bottom {f}"].append(database_part_factor_i_bottom)
for f in factor:
    df_factor_top = pd.concat(df_factor_top_dic[f"Top {f}"])
    df_factor_top = df_factor_top.reset_index(drop=True)
    df_factor_top_dic[f"Top {f}"] = df_factor_top
    df_factor_medium = pd.concat(df_factor_medium_dic[f"Medium {f}"])
    df_factor_medium = df_factor_medium.reset_index(drop=True)
    df_factor_medium_dic[f"Medium {f}"] = df_factor_medium
    df_factor_bottom = pd.concat(df_factor_bottom_dic[f"Bottom {f}"])
    df_factor_bottom = df_factor_bottom.reset_index(drop=True)
    df_factor_bottom_dic[f"Bottom {f}"] = df_factor_bottom

#Sorting portfolio and Apply weighting
reb_comp_dic = {}
#for f in factor:
#    for n in ['top_small_','medium_small_','bottom_small_','top_big_','medium_big_','bottom_big_']:
#        reb_comp_dic[f"{n}{f}"] = []
for f in factor:
    top_small_list = []
    medium_small_list = []
    bottom_small_list = []
    top_big_list = []
    medium_big_list = []
    bottom_big_list = []
    for en,i in enumerate(param_date_list[:-1]):
        stock_top = df_factor_top_dic[f"Top {f}"][['Date','Kode','Market Cap']]
        stock_top_i = stock_top[stock_top['Date']==i]
        stock_medium = df_factor_medium_dic[f"Medium {f}"][['Date','Kode','Market Cap']]
        stock_medium_i = stock_medium[stock_medium['Date']==i]
        stock_bottom = df_factor_bottom_dic[f"Bottom {f}"][['Date','Kode','Market Cap']]
        stock_bottom_i = stock_bottom[stock_bottom['Date']==i]
        stock_small = df_small[['Date','Kode','Market Cap']]
        stock_small_i = stock_small[stock_small['Date']==i]
        stock_big = df_big[['Date','Kode','Market Cap']]
        stock_big_i = stock_big[stock_big['Date']==i]
        stock_top_small = pd.merge(stock_top_i,stock_small_i,how='inner')
        stock_medium_small = pd.merge(stock_medium_i,stock_small_i,how='inner')
        stock_bottom_small = pd.merge(stock_bottom_i,stock_small_i,how='inner')
        stock_top_big = pd.merge(stock_top_i,stock_big_i,how='inner')
        stock_medium_big = pd.merge(stock_medium_i,stock_big_i,how='inner')
        stock_bottom_big = pd.merge(stock_bottom_i,stock_big_i,how='inner')
        stock_top_small = apply_weighting(data=stock_top_small,weighting=weighting)
        stock_medium_small = apply_weighting(data=stock_medium_small,weighting=weighting)
        stock_bottom_small = apply_weighting(data=stock_bottom_small,weighting=weighting)
        stock_top_big = apply_weighting(data=stock_top_big,weighting=weighting)
        stock_medium_big = apply_weighting(data=stock_medium_big,weighting=weighting)
        stock_bottom_big = apply_weighting(data=stock_bottom_big,weighting=weighting)
        stock_top_small['From'] = buy_date_list[:-1][en]
        stock_medium_small['From'] = buy_date_list[:-1][en]
        stock_bottom_small['From'] = buy_date_list[:-1][en]
        stock_top_big['From'] = buy_date_list[:-1][en]
        stock_medium_big['From'] = buy_date_list[:-1][en]
        stock_bottom_big['From'] = buy_date_list[:-1][en]
        stock_top_small['To'] = sell_date_list[en]
        stock_medium_small['To'] = sell_date_list[en]
        stock_bottom_small['To'] = sell_date_list[en]
        stock_top_big['To'] = sell_date_list[en]
        stock_medium_big['To'] = sell_date_list[en]
        stock_bottom_big['To'] = sell_date_list[en]
        stock_top_small['Number'] = en+1
        stock_medium_small['Number'] = en+1
        stock_bottom_small['Number'] = en+1
        stock_top_big['Number'] = en+1
        stock_medium_big['Number'] = en+1
        stock_bottom_big['Number'] = en+1
        top_small_list.append(stock_top_small)
        medium_small_list.append(stock_medium_small)
        bottom_small_list.append(stock_bottom_small)
        top_big_list.append(stock_top_big)
        medium_big_list.append(stock_medium_big)
        bottom_big_list.append(stock_bottom_big)
    for l in [top_small_list,medium_small_list,bottom_small_list,top_big_list,medium_big_list,bottom_big_list]:
        for n in ['top_small_','medium_small_','bottom_small_','top_big_','medium_big_','bottom_big_']:
            reb_comp_dic[f"{n}{f}"] = pd.concat(l).reset_index(drop=True)
            reb_comp_dic[f"{n}{f}"] = reb_comp_dic[f"{n}{f}"][['Kode','Weight','From','To','Number']]


In [221]:
buy_sell_date_list = buy_date_list + sell_date_list
database_price = database[['Kode','Date','Open Price','High Price','Low Price','Close Price']]
database_price = database_price[database_price['Date'].isin(buy_sell_date_list)]
database_price['Kode + Date'] = database_price['Kode'].astype('str') + ' ' + database_price['Date'].astype('str')
reb_comp_act_dic = {}
for f in factor:
    for n in ['top_small_','medium_small_','bottom_small_','top_big_','medium_big_','bottom_big_']:
        reb_comp_act_dic[f"{n}{f}"] = []
for f in factor:
    for n in ['top_small_','medium_small_','bottom_small_','top_big_','medium_big_','bottom_big_']:
        reb_comp = reb_comp_dic[f'{n}{f}']
        start_date = reb_comp['From'].sort_values().astype('str').values[0]
        end_date = reb_comp['To'].sort_values().astype('str').values[-1]
        for i in list(reb_comp['Number'].sort_values().unique()):
            reb_comp_i = reb_comp[reb_comp['Number']==i].reset_index(drop=True)
            stock_list_i = list(reb_comp_i['Kode'])
            if len(stock_list_i) > len(set(stock_list_i)):
                print('Stock list is not unique')
                break
            else:
                pass
            from_date = reb_comp_i['From'].astype('str').unique()[0]
            reb_comp_act_i = pd.DataFrame({'Kode':stock_list_i})
            reb_comp_act_i['Date'] = from_date 
            reb_comp_act_i['Kode + Date'] = reb_comp_act_i['Kode'].astype('str') + ' ' + reb_comp_act_i['Date'].astype('str')
            reb_comp_act_i = reb_comp_act_i.merge(database_price.drop(columns=['Kode','Date']),how='left',on='Kode + Date')
            reb_comp_act_i = reb_comp_act_i.drop(columns='Kode + Date')
            reb_comp_act_i['Action'] = 'BUY'
            reb_comp_act_i = reb_comp_act_i.sort_values(by='Kode').reset_index(drop=True)
            reb_comp_act_dic[f"{n}{f}"].append(reb_comp_act_i)
            to_date = reb_comp_i['To'].astype('str').unique()[0]
            reb_comp_act_i = pd.DataFrame({'Kode':stock_list_i})
            reb_comp_act_i['Date'] = to_date 
            reb_comp_act_i['Kode + Date'] = reb_comp_act_i['Kode'].astype('str') + ' ' + reb_comp_act_i['Date'].astype('str')
            reb_comp_act_i = reb_comp_act_i.merge(database_price.drop(columns=['Kode','Date']),how='left',on='Kode + Date')
            reb_comp_act_i = reb_comp_act_i.drop(columns='Kode + Date')
            reb_comp_act_i['Action'] = 'SELL'
            reb_comp_act_i = reb_comp_act_i.sort_values(by='Kode').reset_index(drop=True)
            reb_comp_act_dic[f"{n}{f}"].append(reb_comp_act_i)
for f in factor:
    for n in ['top_small_','medium_small_','bottom_small_','top_big_','medium_big_','bottom_big_']:
        reb_comp_act_dic[f"{n}{f}"] = pd.concat(reb_comp_act_dic[f"{n}{f}"]).reset_index(drop=True)


In [ ]:
transaction_dic = {}
for f in factor:
    for n in ['top_small_','medium_small_','bottom_small_','top_big_','medium_big_','bottom_big_']:
        transaction_dic[f"{n}{f}"] = []
for f in factor:
    for n in ['top_small_','medium_small_','bottom_small_','top_big_','medium_big_','bottom_big_']:
        reb_comp_act = reb_comp_act_dic[f"{n}{f}"]
        reb_comp = reb_comp_dic[f'{n}{f}']
        day_number = 0
        reb_comp_act_full_day_list = []
        for i in list(reb_comp_act['Date'].sort_values().unique()):
            reb_comp_act_i = reb_comp_act[reb_comp_act['Date']==i]
            reb_comp_act_i['Day Number'] = day_number
            reb_comp_i = reb_comp[reb_comp['From']==i]
            reb_comp_i = reb_comp_i.rename(columns={'From':'Date'})
            reb_comp_i['Kode + Date'] = reb_comp_i['Kode'].astype('str') + ' ' + reb_comp_i['Date'].astype('str') 
            reb_comp_act_s_list = []
            reb_comp_act_s_buy_list = []
            for s in list(reb_comp_act_i['Kode'].sort_values()):
                reb_comp_act_s = reb_comp_act_i[reb_comp_act_i['Kode']==s]
                reb_comp_act_s['Initial Capital'] = 1_000_000_000
                reb_comp_act_s['Reference Price'] = reb_comp_act_s['Close Price']
                reb_comp_act_s['Price/Lot'] = reb_comp_act_s['Reference Price'].astype('float64')*100
                reb_comp_act_s['Kode + Date'] = reb_comp_act_s['Kode'].astype('str') + ' ' + reb_comp_act_s['Date'].astype('str') 
                if reb_comp_act_s['Action'].values[0] == 'BUY':
                    if day_number == 0:
                        reb_comp_act_s['Reference Capital'] = 1_000_000_000
                    else:
                        reb_comp_act_s['Reference Capital'] = reb_comp_act_full_day['Total Capital'].iloc[-1]
                    reb_comp_act_s = reb_comp_act_s.merge(reb_comp_i[['Kode + Date','Weight']],on='Kode + Date',how='left')
                    reb_comp_act_s['Buy Price'] = reb_comp_act_s['Price/Lot']
                    reb_comp_act_s['Sell Price'] = np.nan
                    reb_comp_act_s['Buy Lot'] = reb_comp_act_s['Reference Capital']*reb_comp_act_s['Weight']/reb_comp_act_s['Buy Price']
                    reb_comp_act_s['Sell Lot'] = np.nan
                    reb_comp_act_s['Hold Lot'] = np.nan
                    reb_comp_act_s['Capital Allocation'] = reb_comp_act_s['Reference Capital']*reb_comp_act_s['Weight']
                    reb_comp_act_s['Invested'] = reb_comp_act_s['Buy Price']*reb_comp_act_s['Buy Lot']
                    reb_comp_act_s['Sold'] = np.nan
                    reb_comp_act_s['Market Value'] = reb_comp_act_s['Close Price']*reb_comp_act_s['Buy Lot']*100
                    reb_comp_act_s['Potential P&L'] = reb_comp_act_s['Market Value'] - reb_comp_act_s['Invested']
                    reb_comp_act_s['Realized P&L'] = np.nan
                    reb_comp_act_s['Cash'] = reb_comp_act_s['Capital Allocation'] - reb_comp_act_s['Invested']
                    reb_comp_act_s['ROI'] = np.nan
                    reb_comp_act_s_buy_list.append(reb_comp_act_s)
                elif reb_comp_act_s['Action'].values[0] == 'SELL':
                    reb_comp_act_full_day_s = reb_comp_act_full_day[reb_comp_act_full_day['Kode']==s]
                    reb_comp_act_s['Reference Capital'] = reb_comp_act_full_day['Total Market Value'].iloc[-1]
                    reb_comp_act_s['Reference Price'] = reb_comp_act_s['Close Price']
                    reb_comp_act_s['Weight'] = 0
                    reb_comp_act_s['Buy Price'] = np.nan
                    reb_comp_act_s['Sell Price'] = reb_comp_act_s['Price/Lot']
                    reb_comp_act_s = reb_comp_act_s.merge(reb_comp_act_buy[['Kode','Buy Lot']],on='Kode',how='left')
                    reb_comp_act_s = reb_comp_act_s.rename(columns={'Buy Lot':'Sell Lot'})
                    reb_comp_act_s['Buy Lot'] = np.nan
                    reb_comp_act_s['Hold Lot'] = np.nan
                    reb_comp_act_s['Capital Allocation'] = np.nan
                    reb_comp_act_s = reb_comp_act_s.merge(reb_comp_act_full_day_s[['Kode','Invested']],on='Kode',how='left')
                    reb_comp_act_s['Sold'] = reb_comp_act_s['Sell Price']*reb_comp_act_s['Sell Lot']
                    reb_comp_act_s['Market Value'] = np.nan
                    reb_comp_act_s['Potential P&L'] = np.nan
                    reb_comp_act_s['Realized P&L'] = reb_comp_act_s['Sold']-reb_comp_act_s['Invested']
                    reb_comp_act_s = reb_comp_act_s.merge(reb_comp_act_full_day_s[['Kode','Cash']],on='Kode',how='left')
                    reb_comp_act_s['Cash'] = reb_comp_act_s['Cash'] + reb_comp_act_s['Sold']
                    sold = reb_comp_act_s['Sold'].iloc[0]
                    invested = reb_comp_act_buy[reb_comp_act_buy['Kode']==s]['Invested'].iloc[0]
                    reb_comp_act_s['ROI'] = (sold-invested)/invested
                else: pass
                reb_comp_act_s_list.append(reb_comp_act_s)
            reb_comp_act_full_day = pd.concat(reb_comp_act_s_list)
            reb_comp_act_full_day['Total Invested'] = reb_comp_act_full_day['Invested'].sum()
            reb_comp_act_full_day['Total Market Value'] = reb_comp_act_full_day['Market Value'].sum()
            reb_comp_act_full_day['Total Potential P&L'] = reb_comp_act_full_day['Potential P&L'].sum()
            reb_comp_act_full_day['Total Realized P&L'] = reb_comp_act_full_day['Realized P&L'].sum()
            reb_comp_act_full_day['Total Cash'] = reb_comp_act_full_day['Cash'].sum() 
            reb_comp_act_full_day['Total Capital'] = reb_comp_act_full_day['Total Market Value'] + reb_comp_act_full_day['Total Cash']
            reb_comp_act_full_day['Weight'] = reb_comp_act_full_day['Weight'].fillna(value=reb_comp_act_full_day['Market Value']/reb_comp_act_full_day['Total Market Value'])
            day_number = day_number + 1
            transaction_dic[f"{n}{f}"].append(reb_comp_act_full_day)
            if len(reb_comp_act_s_buy_list)>0:
                reb_comp_act_buy = pd.concat(reb_comp_act_s_buy_list)
for f in factor:
    for n in ['top_small_','medium_small_','bottom_small_','top_big_','medium_big_','bottom_big_']:
        transaction_dic[f"{n}{f}"] = pd.concat(transaction_dic[f"{n}{f}"]).reset_index(drop=True)


